In [ ]:
%%time

%run D:\workspaces\General\notebooks\image-processing\0-0-common.ipynb

In [ ]:
%%time

submission_data: pd.DataFrame = pd.read_parquet("data/text/AskReddit.parquet", engine='pyarrow', filesystem=file_system)

comment_data: pd.DataFrame = pd.read_parquet("data/text/AskReddit.comments.parquet", engine='pyarrow', filesystem=file_system)

display(submission_data.head())

display(comment_data.head())

In [ ]:
%%time

submission_data["submission_id"] = submission_data.apply(lambda row: "t3_" + row["id"], axis=1)

display(submission_data.head())

In [15]:
%%time

trees = []
for submission in tqdm(submission_data.itertuples(), desc="Building trees", total=len(submission_data)):
    submission_root: Node = Node(submission.id, title=submission.title, text=submission.selftext, score=submission.score, author=submission.author)
    comment_for_submission = comment_data[comment_data["parent_id"] == submission.submission_id]
    for comment in comment_for_submission.itertuples():
        comment_node: Node = Node(comment.id, text=comment.body, score=comment.score, author=comment.author)
        if comment.parent_id == 't3_' + comment.submission_id:
            comment_node.parent = submission_root
        else:
            for elem in submission_root.children:
                if 't1_' + elem.name == comment.parent_id:
                    comment_node.parent = elem
                else:
                    continue
    trees.append(submission_root)

Building trees: 100%|██████████| 21127/21127 [20:32<00:00, 17.14it/s] 


CPU times: total: 18min 48s
Wall time: 20min 32s


In [16]:
root = trees[0]
root.show(attr_list = ['text', 'title'])
root.show()
from bigtree import levelorder_iter
result = [(node.name, node.node_path) for node in levelorder_iter(root)]

pd.DataFrame(data=result)

zquyhv [text=, title=what was ruined by rich people?]
├── j10ieuy [text=Every single piece of water with accessible waterfronts.

Edit:  Wow, lots of upvotes.  Going forward, upvotes will be $8.]
├── j1059t7 [text=Housing - used to be the only people buying houses were those that were intending to live in them. Now you have rich people looking into investing in real estate, and guess what rich houses are poor investments. So rich people are buying up all the affordable housing driving up the price and creating scarcity.]
├── j10wjru [text=So many things, but I'll say trucks. Once upon a time, a humble working class vehicle for people who need to be able to do things themselves, now they're all luxury vehicles with massive margins, unaffordable to anyone who needs them to do real work.]
├── j1064zp [text=Everest]
├── j10et9p [text=Old school ski mountains with family vibes and stoner lifties rocking to music.  We always watched those silly “ski patrol” movies about the big corporations 

,0,1
0,zquyhv,"[Node(/zquyhv, author=Lattethecoffeaddict, sco..."
1,j10ieuy,"(Node(/zquyhv, author=Lattethecoffeaddict, sco..."
2,j1059t7,"(Node(/zquyhv, author=Lattethecoffeaddict, sco..."
3,j10wjru,"(Node(/zquyhv, author=Lattethecoffeaddict, sco..."
4,j1064zp,"(Node(/zquyhv, author=Lattethecoffeaddict, sco..."
...,...,...
95,j107wsi,"(Node(/zquyhv, author=Lattethecoffeaddict, sco..."
96,j10cxws,"(Node(/zquyhv, author=Lattethecoffeaddict, sco..."
97,j101npu,"(Node(/zquyhv, author=Lattethecoffeaddict, sco..."
98,j10i5hs,"(Node(/zquyhv, author=Lattethecoffeaddict, sco..."
